# Analyse des interventions non facturées

Ce notebook permet d'analyser les interventions non facturées dans Notion.

In [ ]:
import pandas as pd
import datetime

from src.notion_module.analyse import from_interventions_to_datas
from src.notion_module.core import query_unbilled_entries
from src.notion_module.core import create_invoice_page

In [ ]:
def etape_2 ():
    """
        Cette fonction permet de récupérer les interventions non facturées
        dans Notion et de les transformer en DataFrame pour analyse.
        Pour ignorer le statut de facturation, on peut passer le dernier paramètre à None
    """
    interventions = query_unbilled_entries("2025-01-01", "2025-01-30", None)

    print(interventions)

    datas = from_interventions_to_datas(interventions)

    df = pd.DataFrame(datas)

    # Conversion de la date en datetime
    df['Date de début'] = pd.to_datetime(df['Date de début'])

    # Ajout colonne mois
    df['Mois'] = df['Date de début'].dt.to_period('M')

    # Calcul somme à facturer
    df['Somme à facturer'] = df['Nombre heures'] * df['Tarif horaire']

    return df

dataframes = etape_2()

In [ ]:
def by_city(df):
    df_ville = df.groupby('Ville').agg(
        nb_ecoles=('Ecole', 'nunique'),
        nb_heures=('Nombre heures', 'sum'),
        somme_a_facturer=('Somme à facturer', 'sum')
    ).reset_index()

    print(df_ville)

by_city(dataframes)

In [ ]:
def by_school_and_class(df):
    df_ecole_classe = df.groupby(['Ecole', 'Classe']).agg(
        nb_heures=('Nombre heures', 'sum'),
        somme_a_facturer=('Somme à facturer', 'sum')
    ).reset_index()

    print(df_ecole_classe)

by_school_and_class(dataframes)

In [ ]:
def by_month(df):
    # On distingue passé vs futur (par rapport à aujourd'hui)
    aujourdhui = pd.to_datetime(datetime.datetime.today().date())

    df['periode'] = df['Date de début'].apply(lambda d: 'Passé' if d < aujourdhui else 'Futur')

    df_mois_periode = df.groupby(['Mois', 'periode']).agg(
        nb_heures=('Nombre heures', 'sum')
    ).reset_index()

    print(df_mois_periode)

by_month(dataframes)

In [ ]:
def etape_3():
    """
        Cette fonction permet de créer une page de facture dans Notion
        à partir des données d'interventions non facturées.
    """
    interventions = [
        {'object': 'page', 'id': '2102cdbb-4757-816a-bb5a-f9c59736f959', 'created_time': '2025-06-12T07:23:00.000Z', 'last_edited_time': '2025-06-12T07:23:00.000Z', 'created_by': {'object': 'user', 'id': '2f589f48-fe35-4317-8f84-345e601d4a44'}, 'last_edited_by': {'object': 'user', 'id': '2f589f48-fe35-4317-8f84-345e601d4a44'}, 'cover': None, 'icon': {'type': 'external', 'external': {'url': 'https://www.notion.so/icons/checkmark_green.svg'}}, 'parent': {'type': 'database_id', 'database_id': '2102cdbb-4757-816a-a73e-d3dc1dc16b36'}, 'archived': False, 'in_trash': False, 'properties': {'Ville': {'id': '%3Aso%7C', 'type': 'select', 'select': {'id': 'f19b6eb9-8259-4e8b-bf70-29e154dfac65', 'name': 'Le Kremlin-Bicêtre', 'color': 'default'}}, 'Niveau d’effort': {'id': '%3BoM%7C', 'type': 'select', 'select': {'id': '1993545d-653e-48d0-9496-cfc698410019', 'name': 'Élevé', 'color': 'red'}}, 'Classe': {'id': '%3D%3C%60%40', 'type': 'select', 'select': {'id': 'b4f0bb80-bba0-4b83-886d-33e206db9eb6', 'name': 'M1', 'color': 'gray'}}, 'Date de fin': {'id': 'BC%7BD', 'type': 'date', 'date': None}, 'Total': {'id': 'E%5DAO', 'type': 'formula', 'formula': {'type': 'number', 'number': 70}}, 'Ecole': {'id': 'GDGQ', 'type': 'select', 'select': {'id': '8a6b5b53-1dbd-4449-8c89-419f54def5a0', 'name': 'EPITECH Digital', 'color': 'purple'}}, 'Description': {'id': 'J%3AF%5B', 'type': 'rich_text', 'rich_text': []}, 'Nombre heures': {'id': 'JjGy', 'type': 'number', 'number': 7}, 'Type de tâche': {'id': 'L~%5D%3E', 'type': 'select', 'select': {'id': 'e692647e-d5dd-444e-a582-b43f159c6bcf', 'name': 'Cours présenté', 'color': 'green'}}, 'En souffrance': {'id': 'Ox~q', 'type': 'formula', 'formula': {'type': 'string', 'string': '⏰ Past Due'}}, 'Personne assignée': {'id': 'Qv%5Dw', 'type': 'people', 'people': []}, 'État': {'id': 'S%5E%3Ed', 'type': 'status', 'status': {'id': '2150867b-0a8d-4a4e-9135-850e487ebad7', 'name': 'Terminé', 'color': 'green'}}, 'Facturé': {'id': 'UrM_', 'type': 'checkbox', 'checkbox': False}, 'ID de facture': {'id': 'XMVJ', 'type': 'select', 'select': None}, 'Priorité': {'id': '%5EYGk', 'type': 'select', 'select': {'id': '31c883a7-33f8-470e-9176-8040a40106a4', 'name': 'Élevée', 'color': 'red'}}, 'Tarif horaire': {'id': '%60_Pc', 'type': 'number', 'number': 10}, 'A facturer': {'id': 'dF%3AB', 'type': 'select', 'select': {'id': 'd67cf60e-083f-48a0-a165-0a31c7710868', 'name': 'True', 'color': 'yellow'}}, 'Date de début': {'id': 'hjFQ', 'type': 'date', 'date': {'start': '2025-03-05', 'end': None, 'time_zone': None}}, 'Contenu de cours': {'id': 'rDxR', 'type': 'relation', 'relation': [], 'has_more': False}, 'Cours': {'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': 'Innovation en IA & Transformation des organisations partie 1', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'Innovation en IA & Transformation des organisations partie 1', 'href': None}]}}, 'url': 'https://www.notion.so/Innovation-en-IA-Transformation-des-organisations-partie-1-2102cdbb4757816abb5af9c59736f959', 'public_url': 'https://tin-cloudberry-7b3.notion.site/Innovation-en-IA-Transformation-des-organisations-partie-1-2102cdbb4757816abb5af9c59736f959'},
        {'object': 'page', 'id': '2102cdbb-4757-8176-9f82-ec600369043f', 'created_time': '2025-06-12T07:23:00.000Z', 'last_edited_time': '2025-06-12T07:23:00.000Z', 'created_by': {'object': 'user', 'id': '2f589f48-fe35-4317-8f84-345e601d4a44'}, 'last_edited_by': {'object': 'user', 'id': '2f589f48-fe35-4317-8f84-345e601d4a44'}, 'cover': None, 'icon': {'type': 'external', 'external': {'url': 'https://www.notion.so/icons/checkmark_green.svg'}}, 'parent': {'type': 'database_id', 'database_id': '2102cdbb-4757-816a-a73e-d3dc1dc16b36'}, 'archived': False, 'in_trash': False, 'properties': {'Ville': {'id': '%3Aso%7C', 'type': 'select', 'select': {'id': 'f19b6eb9-8259-4e8b-bf70-29e154dfac65', 'name': 'Le Kremlin-Bicêtre', 'color': 'default'}}, 'Niveau d’effort': {'id': '%3BoM%7C', 'type': 'select', 'select': {'id': '1993545d-653e-48d0-9496-cfc698410019', 'name': 'Élevé', 'color': 'red'}}, 'Classe': {'id': '%3D%3C%60%40', 'type': 'select', 'select': {'id': 'b4f0bb80-bba0-4b83-886d-33e206db9eb6', 'name': 'M1', 'color': 'gray'}}, 'Date de fin': {'id': 'BC%7BD', 'type': 'date', 'date': None}, 'Total': {'id': 'E%5DAO', 'type': 'formula', 'formula': {'type': 'number', 'number': 70}}, 'Ecole': {'id': 'GDGQ', 'type': 'select', 'select': {'id': '8a6b5b53-1dbd-4449-8c89-419f54def5a0', 'name': 'EPITECH Digital', 'color': 'purple'}}, 'Description': {'id': 'J%3AF%5B', 'type': 'rich_text', 'rich_text': []}, 'Nombre heures': {'id': 'JjGy', 'type': 'number', 'number': 7}, 'Type de tâche': {'id': 'L~%5D%3E', 'type': 'select', 'select': {'id': 'e692647e-d5dd-444e-a582-b43f159c6bcf', 'name': 'Cours présenté', 'color': 'green'}}, 'En souffrance': {'id': 'Ox~q', 'type': 'formula', 'formula': {'type': 'string', 'string': '⏰ Past Due'}}, 'Personne assignée': {'id': 'Qv%5Dw', 'type': 'people', 'people': []}, 'État': {'id': 'S%5E%3Ed', 'type': 'status', 'status': {'id': '2150867b-0a8d-4a4e-9135-850e487ebad7', 'name': 'Terminé', 'color': 'green'}}, 'Facturé': {'id': 'UrM_', 'type': 'checkbox', 'checkbox': False}, 'ID de facture': {'id': 'XMVJ', 'type': 'select', 'select': None}, 'Priorité': {'id': '%5EYGk', 'type': 'select', 'select': {'id': '31c883a7-33f8-470e-9176-8040a40106a4', 'name': 'Élevée', 'color': 'red'}}, 'Tarif horaire': {'id': '%60_Pc', 'type': 'number', 'number': 10}, 'A facturer': {'id': 'dF%3AB', 'type': 'select', 'select': {'id': 'd67cf60e-083f-48a0-a165-0a31c7710868', 'name': 'True', 'color': 'yellow'}}, 'Date de début': {'id': 'hjFQ', 'type': 'date', 'date': {'start': '2025-03-05', 'end': None, 'time_zone': None}}, 'Contenu de cours': {'id': 'rDxR', 'type': 'relation', 'relation': [], 'has_more': False}, 'Cours': {'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': 'Innovation en IA & Transformation des organisations partie 1', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'Innovation en IA & Transformation des organisations partie 1', 'href': None}]}}, 'url': 'https://www.notion.so/Innovation-en-IA-Transformation-des-organisations-partie-1-2102cdbb475781769f82ec600369043f', 'public_url': 'https://tin-cloudberry-7b3.notion.site/Innovation-en-IA-Transformation-des-organisations-partie-1-2102cdbb475781769f82ec600369043f'}
    ]

    # On crée la page de facture
    print(create_invoice_page("nexa", interventions, "1000", "C0001"))

etape_3()